In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install laspy

In [ ]:
!unzip "/content/drive/MyDrive/class_0_500_samples.zip" -d "/content"

Archive:  /content/drive/MyDrive/class_0_500_samples.zip
   creating: /content/class_0_500_samples/
  inflating: /content/class_0_500_samples/cloud55044bb585229f81_Block_44_tile_0_label0.las  
  inflating: /content/class_0_500_samples/cloud55044bb585229f81_Block_44_tile_100_label0.las  
  inflating: /content/class_0_500_samples/cloud55044bb585229f81_Block_44_tile_101_label0.las  
  inflating: /content/class_0_500_samples/cloud55044bb585229f81_Block_44_tile_102_label0.las  
  inflating: /content/class_0_500_samples/cloud55044bb585229f81_Block_44_tile_103_label0.las  
  inflating: /content/class_0_500_samples/cloud55044bb585229f81_Block_44_tile_104_label0.las  
  inflating: /content/class_0_500_samples/cloud55044bb585229f81_Block_44_tile_105_label0.las  
  inflating: /content/class_0_500_samples/cloud55044bb585229f81_Block_44_tile_106_label0.las  
  inflating: /content/class_0_500_samples/cloud55044bb585229f81_Block_44_tile_108_label0.las  
  inflating: /content/class_0_500_samples/cloud

In [ ]:
import os

las_dir = "/content/class_0_500_samples"
las_files = [f for f in os.listdir(las_dir) if f.endswith('.las')] # List all LAS files in the directory
print(f"Number of LAS files in {las_dir}: {len(las_files)}")

Number of LAS files in /content/class_0_500_samples: 500


In [ ]:
import os
import numpy as np
import laspy
import matplotlib.pyplot as plt

# --- Step 1: Load LAS file ---
def load_las(file_path):
    with laspy.open(file_path) as fh:
        data = fh.read()
        coords = np.vstack((data.x, data.y, data.z)).T

        # Check if RGB attributes exist
        has_rgb = hasattr(data, "red") and hasattr(data, "green") and hasattr(data, "blue")
        if has_rgb:
            rgb = np.vstack((data.red, data.green, data.blue)).T.astype(np.float64)
        else:
            rgb = None

    return coords, rgb


# --- Step 2: Compute Q matrix ---
def compute_Q(points, d):
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    if x_max == x_min:
        x_norm = np.zeros_like(points[:, 0])
    else:
        x_norm = (points[:, 0] - x_min) / (x_max - x_min)
    if y_max == y_min:
        y_norm = np.zeros_like(points[:, 1])
    else:
        y_norm = (points[:, 1] - y_min) / (y_max - y_min)
    xi = np.floor(x_norm * d).astype(int)
    yi = np.floor(y_norm * d).astype(int)
    xi = np.clip(xi, 0, d - 1)
    yi = np.clip(yi, 0, d - 1)
    Q = np.zeros((d, d), dtype=np.float64)
    counts = np.zeros((d, d), dtype=np.int32)
    for x, y, z in zip(xi, yi, points[:, 2]):
        Q[y, x] += z
        counts[y, x] += 1
    mask = counts > 0
    Q[mask] = Q[mask] / counts[mask]
    return Q, mask


# --- Step 3: Compute V1 (Elevation Variability) ---
def compute_V1(Q):
    d = Q.shape[0]
    q_bar = np.sum(Q) / (d * d)
    v1 = np.sum((Q - q_bar) ** 2) / (d ** 2 - 1)
    return v1


# --- Step 4: Compute V2 (Directional Variability) ---
def compute_V2(Q, mask=None, use_circular=False, epsilon=1e-10):
    d = Q.shape[0]
    if d < 2:
        return np.nan

    dQdx = np.zeros_like(Q)
    dQdy = np.zeros_like(Q)

    if d > 2:
        dQdx[:, 1:-1] = (Q[:, 2:] - Q[:, :-2]) / 2.0
        dQdy[1:-1, :] = (Q[2:, :] - Q[:-2, :]) / 2.0

    dQdx[:, 0] = Q[:, 1] - Q[:, 0]
    dQdx[:, -1] = Q[:, -1] - Q[:, -2]
    dQdy[0, :] = Q[1, :] - Q[0, :]
    dQdy[-1, :] = Q[-1, :] - Q[-2, :]

    mag = np.sqrt(dQdx ** 2 + dQdy ** 2)
    grad_mask = mag > 1e-8
    if mask is not None:
        theta_mask = np.logical_and(grad_mask, mask)
    else:
        theta_mask = grad_mask

    theta = np.arctan2(dQdy, dQdx)
    theta = np.ma.masked_where(~theta_mask, theta)

    if np.ma.count(theta) < 2:
        return np.nan

    theta = theta.compressed()
    if use_circular:
        mean_sin = np.mean(np.sin(theta))
        mean_cos = np.mean(np.cos(theta))
        R = np.sqrt(mean_cos ** 2 + mean_sin ** 2)
        var = 1 - R
        var *= (np.pi ** 2 / 3)
    else:
        var = np.var(theta, ddof=1)

    v2 = np.log(var + epsilon)
    return v2


# --- Step 5: Compute RGB features ---
def compute_RGB(rgb):
    if rgb is None:
        return np.nan, np.nan, np.nan, np.nan
    r_mean = np.mean(rgb[:, 0])
    g_mean = np.mean(rgb[:, 1])
    b_mean = np.mean(rgb[:, 2])
    rgb_mean = (r_mean + g_mean + b_mean) / 3
    return r_mean, g_mean, b_mean, rgb_mean

In [ ]:
import os
import pandas as pd
import zipfile

# --- Parameters ---
output_zip = "/content/features_all_grids00.zip"

# --- Create a temporary folder to store CSVs ---
output_folder = "/content/features_per_grid00"
os.makedirs(output_folder, exist_ok=True)

print("V1,V2,R_mean,G_mean,B_mean,RGB_mean")

# --- Loop through grid resolutions ---
for d in range(2, 36):  # Grid resolution 2 → 35
    print(f"\n--- Processing grid resolution: {d} ---")
    results = []

    for las_file in os.listdir(las_dir):
        if not las_file.endswith(".las"):
            continue

        file_path = os.path.join(las_dir, las_file)
        try:
            points, rgb = load_las(file_path)
            Q, mask = compute_Q(points, d)
            v1 = compute_V1(Q)
            v2 = compute_V2(Q, mask=mask, use_circular=True)
            r_mean, g_mean, b_mean, rgb_mean = compute_RGB(rgb)

            # Append only numeric results
            results.append({
                'V1': v1,
                'V2': v2,
                'R_mean': r_mean,
                'G_mean': g_mean,
                'B_mean': b_mean,
                'RGB_mean': rgb_mean,
                'Class': 0  # Replace as needed: 0=sea, 1=forest, 2=beach
            })

            print(f"{v1:.8f},{v2:.8f},{r_mean:.2f},{g_mean:.2f},{b_mean:.2f},{rgb_mean:.2f},0")

        except Exception as e:
            print(f" Error processing {las_file} at grid {d}: {e}")

    # --- Save one CSV per grid resolution ---
    if results:
        df = pd.DataFrame(results)
        csv_path = os.path.join(output_folder, f"features_d{d}.csv")
        df[['V1', 'V2', 'R_mean', 'G_mean', 'B_mean', 'RGB_mean', 'Class']].to_csv(
            csv_path, index=False, header=False
        )
        print(f" Saved: {csv_path}")

# --- Zip all CSVs together for one-time download ---
with zipfile.ZipFile(output_zip, "w") as zipf:
    for file in os.listdir(output_folder):
        zipf.write(os.path.join(output_folder, file), arcname=file)

print(f"\n All CSV files zipped to {output_zip}")

V1,V2,R_mean,G_mean,B_mean,RGB_mean

--- Processing grid resolution: 31 ---
0.02840550,1.03549244,30839.28,33098.10,28569.98,30835.79,0
0.14021681,1.15982227,22731.11,31145.86,21406.38,25094.45,0
0.00966656,1.14024442,33486.54,33122.88,30189.04,32266.15,0
0.00568071,1.09886998,31447.90,33409.73,28356.65,31071.43,0
0.02036512,1.09571301,26192.98,29611.61,24456.51,26753.70,0
0.02271876,1.15323151,22780.60,31564.74,21988.77,25444.71,0
0.13666378,1.18979358,21651.92,31309.94,21194.31,24718.72,0
0.15409519,1.13667877,23266.98,32056.82,21716.01,25679.94,0
0.05875739,1.14105489,22647.18,32017.16,21973.75,25546.03,0
0.12271612,1.15947959,22405.88,31962.19,22060.76,25476.28,0
0.00399982,0.91195969,25707.10,29130.94,24269.96,26369.33,0
0.00105567,1.13346027,21671.10,30228.03,21022.92,24307.35,0
0.14218644,1.17588814,23364.50,32078.40,21755.94,25732.95,0
0.14318953,1.15356291,27316.77,30263.98,25225.63,27602.12,0
0.14308530,1.15736254,22582.67,31322.51,22297.99,25401.05,0
0.08140703,1.14274738,26